In [32]:
import sqlite3
con = sqlite3.connect("/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/log_p2p_median_dirichlet.db")
cur = con.cursor()


In [ ]:
cur.execute("CREATE TABLE ddpm(experiment_name, checkpoint_name, smoothing, dataset_name, split, dirichlet, p2p_median, zoomout_mean, zoomout_median, pred_mean, pred_median)")

In [11]:
con.close()

In [34]:
import torch

err_list = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32)

In [36]:
data = [
    ("test", 'chkpt_99', 'no', 'FAUST_orig_pair', 'test',
     10, 15,
     err_list.mean().item(), err_list.median().item(),
     err_list.mean().item(), err_list.median().item()),
]
cur.executemany("INSERT INTO ddpm VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data)
con.commit() 

In [ ]:
cur.execute("SELECT * FROM ddpm").fetchall()

In [47]:
con.close()

In [28]:
import pandas as pd
import sqlite3

pd.set_option('display.max_rows', 500)

with sqlite3.connect("/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/log_p2p_median_dirichlet.db") as con:
    df = pd.read_sql_query("SELECT * FROM ddpm", con)
    
df.sort_values(['dataset_name', 'p2p_median', 'experiment_name'], ascending=True, inplace=True)

# for each unique dataset_name, show 5 best experiments
# df.groupby('dataset_name').apply(lambda x: x.nsmallest(8, 'p2p_median')).reset_index(drop=True)

# for each unique dataset_name, show 3 best experiments with checkpoint_name = 'epoch_99' and 3 best experiments with checkpoint_name = 'checkpoint_99.pt'
# df_epoch_99 = df[df['checkpoint_name'] == 'epoch_99'].copy()
# df_checkpoint_99 = df[df['checkpoint_name'] == 'checkpoint_99.pt'].copy()

# df_epoch_99.groupby('dataset_name').apply(lambda x: x.nsmallest(3, 'p2p_median')).reset_index(drop=True)
# df_checkpoint_99.groupby('dataset_name').apply(lambda x: x.nsmallest(3, 'p2p_median')).reset_index(drop=True)

# # combine both dataframes
# df_combined = pd.concat([df_epoch_99, df_checkpoint_99])

# df_combined.sort_values(['dataset_name', 'p2p_median', 'experiment_name'], ascending=True, inplace=True)

# df_combined

# round all numbers  to 2 decimal places
df = df.round(2)

# only keep rows with '24' in the experiment_name
# df = df[df['experiment_name'].str.contains('48') | (df['experiment_name'] == 'single_template_remeshed')]

# highlight the rows with 'single_template_remeshed_augShapes' in the experiment_name
df.style.apply(lambda x: ['background: lightgreen' if x['experiment_name'].str.contains('single_template_remeshed_augShapes') else '' for i in x], axis=1)

df

In [26]:
# save the dataframe to a csv file
df.to_csv('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/log_p2p_median_dirichlet.csv', index=False)

In [ ]:
len(df.index)

In [44]:
# cur.executemany(
#     "SELECT * FROM ddpm WHERE experiment_name=? AND checkpoint_name=? AND smoothing=? AND dataset_name=? AND split=?",
#     [('pair_10_xy_distributed',
#       'epoch_99',
#       'no',
#       'FAUST_a_pair',
#       'test'
#       )]).fetchone()

if cur.execute(f"SELECT * FROM ddpm WHERE experiment_name='pair_10_xy_distributed' AND checkpoint_name='epoch_99' AND dataset_name='FAUST_orig_pair' AND split='test' AND smoothing='no'").fetchall():
    cur.execute(f"DELETE FROM ddpm WHERE experiment_name='pair_10_xy_distributed' AND checkpoint_name='epoch_99' AND dataset_name='FAUST_orig_pair' AND split='test' AND smoothing='no'")
        

In [62]:
con.close()

In [ ]:
l = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
l[:int(round(1.6, 0))]

In [ ]:
# get all files from /home/s94zalek_hpc/shape_matching/data/SHREC19_r/corres
import os

files = os.listdir("/home/s94zalek_hpc/shape_matching/data/SHREC19_r/corres")

files[290]

In [105]:
# read each yaml file in /home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results
import yaml
import os
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 1000)


base_dir = "/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm_results"
# logs_folder_name = "logs_robustMedian_fixedSmoothing_64_10_0.2"
logs_folder_name = "logs_robustMedian_fixedSmoothing_128_16_0.2"

# logs_folder_name = "test_partial_on_train_data"
# logs_folder_name = "test_partial_on_train_data_1_0.5_0.9"
# logs_folder_name='logs_templateZoomout'
# logs_folder_name='logs_partial_symm'
# logs_folder_name='logs_partial_symm_zo'

yaml_files = os.listdir(f"{base_dir}/{logs_folder_name}")
yaml_data = []

for file in yaml_files:
    with open(f"{base_dir}/{logs_folder_name}/{file}", 'r') as f:
        yaml_data.append(yaml.load(f, Loader=yaml.FullLoader))

# convert yaml_data to a pandas dataframe
df = pd.DataFrame(yaml_data)
df.sort_values(['dataset_name', 'median_pairzo', 'experiment_name'], ascending=True, inplace=True)

# remove entries with dataset_name = 'DT4D_inter_pair' and 'DT4D_intra_pair'
df = df[(df['dataset_name'] != 'DT4D_inter_pair') & (df['dataset_name'] != 'DT4D_intra_pair')]

if 'smoothing_type' in df.columns:
    df = df[(df['smoothing_type'] != 'taubin')]
if 'smoothing' in df.columns:
    df = df[(df['smoothing'] != 'taubin-5')]

# df.sort_values(['dataset_name', 'geo_err_est_rev_mean', 'experiment_name'], ascending=True, inplace=True)

df = df.round(2)

df


In [ ]:
# get current time in readable format, for use as a filename
import datetime

(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), )
